<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/videos_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload() # here upload 'chrome-cookies.txt'

In [ ]:
!pip install -q yt_dlp

In [ ]:
import pandas as pd
import cv2
import requests
import os
from google.colab.patches import cv2_imshow
from tqdm import tqdm

In [ ]:
import yt_dlp

In [ ]:
# load the JSON data
js_file = pd.read_json("/content/drive/MyDrive/NLP/WLASL_v0.3.json")
folder = "/content/drive/MyDrive/NLP/dataset/"

training_folder = folder + "train/"
test_folder = folder + "test/"
training_video = training_folder + "video/"
test_video = test_folder + "video/"
training_images = training_folder + "images/"
test_images = test_folder + "images/"
os.makedirs(training_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)
os.makedirs(training_video, exist_ok=True)
os.makedirs(test_video, exist_ok=True)
os.makedirs(training_images, exist_ok=True)
os.makedirs(test_images, exist_ok=True)

In [ ]:
# sources:
# {'asl5200',     youtube.com
# 'aslbrick',     mp4
# 'asldeafined',  mp4
# 'asllex',       youtu.be
# 'aslpro',       swf
# 'aslsearch',    mp4
# 'aslsignbank',  mp4
# 'aslu',         youtube.com
# 'elementalasl', mov (like mp4 maybe?)
# 'handspeak',    mp4
# 'lillybauer',   youtube.com
# 'nabboud',      youtube.com
# 'northtexas',   youtube.com
# 'scott',        youtube.com
# 'signingsavvy', mp4
# 'signschool',   mp4
# 'spreadthesign',mp4
# 'startasl',     mp4
# 'valencia-asl'} youtube.com

In [ ]:
# download youtube videos

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']
not_downloaded = 0
tot_youtube_videos = 0
downloaded = []

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    filename = f"{word}_{video_id}.mp4"

    if source in youtube_videos:
      tot_youtube_videos += 1
      output_path = training_video if split == 'train' else test_video
      dest_path = os.path.join(output_path, filename)
      if 'youtu.be/' in url:
          continue

      if os.path.exists(dest_path):
        downloaded.append(video_id)
        continue

      ydl_opts = {
        'outtmpl': dest_path,
        'format': 'bestvideo+bestaudio/best',
        'merge_output_format': 'mp4',
        'quiet': True,
        'sleep_interval_requests': 2,
        'max_sleep_interval': 5,
        'cookies': 'chrome-cookies.txt'
        }
      try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
          ydl.download([url])
          downloaded.append(video_id)
          #print(video_id, 'DOWNLOADED')
      except:
        #print(video_id)
        not_downloaded += 1
        continue

In [ ]:
# download non-youtube videos (mp4 or mov)
# 9112 non-youtube videos

youtube_videos = ['asl5200', 'asllex', 'aslu', 'lillybauer', 'nabboud', 'northtexas', 'scott', 'valencia-asl']

for i, word in enumerate(tqdm(list(js_file['gloss']), desc='glosses')):
  for j, instance in enumerate(js_file['instances'][i]):

    video_id = js_file['instances'][i][j]['video_id']
    url = js_file['instances'][i][j]['url']
    source = js_file['instances'][i][j]['source']
    split = js_file['instances'][i][j]['split']
    filename = f"{word}_{video_id}.mp4"

    if source not in youtube_videos:

      output_path = training_video if split == 'train' else test_video
      dest_path = os.path.join(output_path, filename)

      # remove already downloaded but damaged videos
      if os.path.exists(dest_path):
        with open(dest_path, 'rb') as f:
                header = f.read()
                if b'<html' in header.lower() or b'403 Forbidden' in header:
                  os.remove(dest_path)

      try:
        resp = requests.get(url, stream=True)
        # do not download damaged videos
        if resp.status_code != 200:
                    continue
        # save downloaded videos
        with open(dest_path, mode='wb') as f:
            f.write(resp.content)
      except:
        continue